In [1]:
'''
Force start Nvidia GPU: 
CUDA_VISIBLE_DEVICES=0 ollama run gemma2:2b
'''

import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.llms import Ollama
from langchain_core.documents import Document
from langchain.graphs import Neo4jGraph

llm = Ollama(model="gemma2:2b")

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris. 
"""

In [42]:
documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [43]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Relationship'), Node(id='University of Paris', type='Institution'), Node(id='Pierre Curie', type='Person'), Node(id='Nobel Prize', type='Prize'), Node(id='Radioactivity', type='Field of study'), Node(id='Person to win a Nobel Prize twice', type='Achievement'), Node(id='Marie Curie', type='Person'), Node(id='Woman to win a Nobel Prize', type='Achievement')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Woman to win a Nobel Prize', type='Achievement'), type='WAS_THE_FIRST'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Radioactivity', type='Field of study'), type='CONDUCTED_RESEARCH'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Person to win a Nobel Prize twice', type='Achievement'), type='WAS_THE_FIRST'), Relationship(source=Node(id='Pierre Curie', type='Person'), target=Node(id='Nobel Prize', type='Prize'), type='CO-WINNER'), Relationship(source=

To configure apoc: add lines in these files
```
sudo vim /etc/neo4j/apoc.conf
sudo vim /etc/neo4j/neo4j.conf
```

sudo systemctl status neo4j
restart
stop
start


To dump the database
```
sudo neo4j-admin database dump neo4j --to-path=.
```

To load the database (https://neo4j.com/labs/apoc/4.1/export/)
```
sudo neo4j-admin database load neo4j --from-path=. --overwrite-destination=true
```

To use cypher terminal:
```
cypher-shell -u neo4j -p 05673049
```

How to switch bw databases: \
https://community.neo4j.com/t/create-multiple-databases-in-community-version/5025/4

In [2]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "05673049"
graph = Neo4jGraph()

In [61]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

### Visualize the stored graph nodes

In [3]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget

default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"
def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    )
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='500px', width='100%'))

### Querying the Graph Database

In [4]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain

CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to query a graph database.
...
"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers.
...
"""
CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

graph_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph, 
    cypher_prompt=CYPHER_GENERATION_PROMPT, 
    qa_prompt=CYPHER_QA_PROMPT,
    verbose=True
)


In [ ]:
question = """Who is Marie Curie?"""
graph_chain.run(question)